In [81]:
import polars as pl
from functools import partial
from itertools import product
import multiprocessing
from numba import jit
from collections import Counter
import numpy as np

In [3]:
#Get dataframe of all tickers
constituents = pl.read_csv("../Data/SP500/Constituents.csv", infer_schema_length=False)

#Create list of individual stocks from names from dataframe
stockComponents=[]
for k in range(len(constituents)):
    stockComponents.append(constituents[k,0])
    #stockComponents.append(constituents[k,0]+'.L')

In [4]:
#Load prices and timestamps from CSVs, just AAPL for now
currentDf=pl.read_csv("../Data/SP500/minuteHist2021/tradingHours/AAPL.csv", infer_schema_length=None)
openPrices=currentDf['open'].to_numpy()
closePrices=currentDf['close'].to_numpy()
timestamps=currentDf['time'].to_numpy()

In [5]:
#Function for sma
#Using numba for speed
@jit(nopython=True)
def sma(length, date, dataframe):
    start=0-(length+date)-1
    end=0-date-1

    total = 0
    #Simple mean by add and divide
    for k in range(start,end):
        total+=int(dataframe[k])
    average = total/length

    return (average)

In [128]:
#Numba for speed
@jit(nopython=True)
#Takes in list of paramaters and 3 separate arrays, open, close, time
def algo(paramsList, openPrices, closePrices, timestamps):

    change=paramsList[0] #Percent change in price to look for
    takeProfit=paramsList[1] #The take profit percentage
    stopLoss=paramsList[2] #The stop loss percentage
    timeout=paramsList[3] #The timeout when it takes too long
    fee=paramsList[4] #Fee on top of every transaction
    ticksBefore=paramsList[5] #Price point beforehand to look at
    
    tradePercent=0

    total=0

    start=60 #Start of data
    end=int(len(openPrices)-61) #End of data

    for j in range(start, end):

        #Check that its not within the first or last hour of trading
        if timestamps[j]-timestamps[j-60]<4500000 and timestamps[j+60]-timestamps[j]<4500000:

            #If price now is above price before
            if openPrices[j]>openPrices[j-ticksBefore]*(change):

                #Add to tally
                total+=1
                #Set current price bought for
                buyPrice = openPrices[j]

                #Reset time and change so far
                time=0
                returnValue=1

                datapoint=j

                #Iterate until timeout or stop loss or profit taken
                while time < timeout and stopLoss<returnValue<takeProfit:
                    time = datapoint-j
                    currentPrice=closePrices[datapoint]
                    returnValue = currentPrice/buyPrice
                    datapoint+=1
                tradePercent+=returnValue-fee
        
    try:
        tradePercent=tradePercent/total
    except:
        pass
    #Return the average trade percent, the total number of times, and the parameters
    returnList=[tradePercent, total, paramsList[0], paramsList[1], paramsList[2], paramsList[3], paramsList[4], paramsList[5]]
    return returnList
    # print(tradePercent)
    # print(tradePerDay)

#algo with all the dataframes already passed through
partialAlgo = partial(algo, openPrices=openPrices, closePrices=closePrices, timestamps=timestamps)

In [130]:
#change
minimum=1
maximum=1.03
difference=0.005
changeList=np.arange(minimum,maximum,difference)

#takeProfit
minimum=1.005
maximum=1.05
difference=0.005
takeProfitList=np.arange(minimum,maximum,difference)

#stopLoss
minimum=0.95
maximum=0.99
difference=0.005
stopLossList=np.arange(minimum,maximum,difference)

#timeout
timeoutList=[5,10,20,30,50]

#daysBefore
ticksBeforeList=[1,2,3,5,10,20,50]

#minimum total for algo
minTotal=200

#rough commission fee
fee=[0.00]

#Every combination of parameters
fullCombinations=list(product(changeList, takeProfitList, stopLossList, timeoutList, fee, ticksBeforeList))
len(fullCombinations)

22050

In [131]:
#Test algo to compile it
algo(fullCombinations[0], openPrices, closePrices, timestamps)
#fullCombinations[0]

[0.9999803599213265, 165048.0, 1.0, 1.005, 0.95, 5.0, 0.0, 1.0]

In [127]:
#List of results to store in right order
results=[]
orderCombinations=[]

#Check if any params are useless
removechange=[]
removeprofit=[]
removeloss=[]
removetime=[]
removeticks=[]

#Multiprocessing!!! Applies the full combinations list over the algo
with multiprocessing.Pool(40) as pool:
    for result in pool.map(
        partialAlgo,
        fullCombinations
    ):
        #Check if the total tally is high enough to be reasonable
        if result[1]>=minTotal:
            results.append(result[0])
            orderCombinations.append((result[2],result[3],result[4],int(result[5]),result[6],int(result[7])))
        
        #if not then add to useless params list
        elif result[1]<100:
            removechange.append(result[2])
            removeprofit.append(result[3])
            removeloss.append(result[4])
            removetime.append(result[5])
            removeticks.append(result[7])

In [121]:
#Sort results with largest first, sorts the combinations by the same order
sortedResults, resultCombinations = zip(*sorted(zip(results, orderCombinations), reverse=True))
partialAlgo(resultCombinations[0])

[1.001989771913315, 576.0, 1.005, 1.0249999999999995, 0.96, 50.0, 0.0, 2.0]

In [122]:
#Print 10 best parameter lists
for k in range(10):
    print(partialAlgo(resultCombinations[k]))

[1.001989771913315, 576.0, 1.005, 1.0249999999999995, 0.96, 50.0, 0.0, 2.0]
[1.0019793000119084, 576.0, 1.005, 1.0249999999999995, 0.95, 50.0, 0.0, 2.0]
[1.0019777419363376, 576.0, 1.005, 1.0249999999999995, 0.955, 50.0, 0.0, 2.0]
[1.0019723790129331, 576.0, 1.005, 1.0249999999999995, 0.965, 50.0, 0.0, 2.0]
[1.001944714358807, 576.0, 1.005, 1.0349999999999993, 0.96, 50.0, 0.0, 2.0]
[1.0019342424574005, 576.0, 1.005, 1.0349999999999993, 0.95, 50.0, 0.0, 2.0]
[1.0019326843818295, 576.0, 1.005, 1.0349999999999993, 0.955, 50.0, 0.0, 2.0]
[1.0019274504932614, 576.0, 1.005, 1.0399999999999991, 0.96, 50.0, 0.0, 2.0]
[1.001927321458425, 576.0, 1.005, 1.0349999999999993, 0.965, 50.0, 0.0, 2.0]
[1.0019169785918547, 576.0, 1.005, 1.0399999999999991, 0.95, 50.0, 0.0, 2.0]


In [123]:
#Show how many times each parameter was useless
print(Counter(removechange),Counter(removeprofit),Counter(removeloss),Counter(removetime),Counter(removeticks))

Counter({1.0249999999999995: 2700, 1.0299999999999994: 2700, 1.0199999999999996: 2250, 1.0149999999999997: 1800, 1.0099999999999998: 1350}) Counter({1.005: 1080, 1.0099999999999998: 1080, 1.0149999999999997: 1080, 1.0199999999999996: 1080, 1.0249999999999995: 1080, 1.0299999999999994: 1080, 1.0349999999999993: 1080, 1.0399999999999991: 1080, 1.044999999999999: 1080, 1.049999999999999: 1080}) Counter({0.95: 1200, 0.955: 1200, 0.96: 1200, 0.965: 1200, 0.97: 1200, 0.975: 1200, 0.98: 1200, 0.985: 1200, 0.99: 1200}) Counter({5.0: 2160, 10.0: 2160, 20.0: 2160, 30.0: 2160, 50.0: 2160}) Counter({1.0: 2250, 2.0: 2250, 3.0: 2250, 5.0: 1800, 10.0: 1350, 20.0: 900})
